In [16]:
import pandas as pd

data = pd.read_csv("Emotion_classify_Data.csv")

In [17]:
from gensim.models import Word2Vec

import gensim.downloader as api

#wva = api.load('word2vec-google-news-300')

In [18]:

import spacy

nlp=spacy.load("en_core_web_sm")

def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)    
    return filtered_tokens


data["Comment"] = data['Comment'].apply(preprocess) 


data.Comment

corpus = [data["Comment"][i] for i in range(0,len(data))]
    

In [19]:
vector_s = 300

model=Word2Vec(sentences = corpus ,epochs =1000  , window=5, vector_size=vector_s, min_count=5, workers=4)

In [20]:
model.wv.similar_by_word('bad')

[('like', 0.23737168312072754),
 ('distract', 0.1942511796951294),
 ('easy', 0.1718028485774994),
 ('dead', 0.1657799482345581),
 ('carry', 0.16488607227802277),
 ('today', 0.16416817903518677),
 ('chest', 0.16090591251850128),
 ('awesome', 0.15755659341812134),
 ('aspect', 0.15385007858276367),
 ('weak', 0.1493191123008728)]

In [21]:
model.save("model")

In [22]:
import numpy as np 

def avg_word2vec(doc):  
        return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)
               

In [49]:
X_avg = []
for sentence in data["Comment"]:
    X_avg.append(avg_word2vec(sentence))

c:\Users\PC1\Desktop\NLP\RNNv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\PC1\Desktop\NLP\RNNv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [26]:
df=pd.DataFrame()
for i in range(0,len(X_avg)):
    df= pd.concat([pd.DataFrame(X_avg[i].reshape(1,-1)), df] , ignore_index=True  )

C:\Users\PC1\AppData\Local\Temp\ipykernel_4288\1501679086.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df= pd.concat([pd.DataFrame(X_avg[i].reshape(1,-1)), df] , ignore_index=True  )


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5937 entries, 0 to 5936
Columns: 300 entries, 0 to 299
dtypes: float32(300)
memory usage: 6.8 MB


### Target preprocess

In [28]:
import keras

target= data[["Emotion"]]

target = target.replace({"fear" : 0 , "anger" :1 , "joy":2})

target = target.astype(int)

target = keras.utils.to_categorical(target.Emotion, num_classes=3)

C:\Users\PC1\AppData\Local\Temp\ipykernel_4288\1359161491.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  target = target.replace({"fear" : 0 , "anger" :1 , "joy":2})


### Split test/train 

In [61]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(df,target,test_size=0.20)


### ANN classifier

In [62]:
from keras_tuner.tuners import RandomSearch

In [64]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten())
    
    for i in range(hp.Int("num_layers", 1, vector_s)):
        model.add(
            keras.layers.Dense(
                
                units=hp.Int(f"units_{i}", min_value=10, max_value=500, step=5),
                activation=hp.Choice("activation", ["relu","leaky_relu"]),
            )
        )
    if hp.Boolean("dropout"):
        model.add(keras.layers.Dropout(rate=0.25))
    model.add(keras.layers.Dense(3, activation="softmax"))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=["categorical_accuracy"],
    )
    return model

In [65]:
tuner = RandomSearch(
    hypermodel=build_model,
    objective="categorical_accuracy",
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
    directory="classifier",
    project_name="WV",
)

In [66]:
stop_early = keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)

In [67]:
tuner.search(X_train,y_train ,epochs=100,validation_data=(X_test,y_test) , callbacks=[stop_early])

Trial 10 Complete [00h 01m 26s]
categorical_accuracy: 0.3284901976585388

Best categorical_accuracy So Far: 0.3407033085823059
Total elapsed time: 00h 18m 16s


In [ ]:
#Best model
models = tuner.get_best_models(num_models=1)
best_model = models[0]
best_model.summary()

C:\Users\PC1\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 300)            │         3,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 170)            │        51,170 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 310)            │        53,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 40)             │        12,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 165)            │         6,765 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 475)            │        78,850 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 3)              │         1,428 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 206,963 (808.45 KB)

 Trainable params: 206,963 (808.45 KB)

 Non-trainable params: 0 (0.00 B)

In [68]:
best_hps = tuner.get_best_hyperparameters(5)

model_clf = build_model(best_hps[0])

model_clf.fit(x=X_train, y=y_train, validation_split=0.2 ,epochs=50)

Epoch 1/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 77s 109ms/step - categorical_accuracy: 0.3210 - loss: 1.0988 - val_categorical_accuracy: 0.3547 - val_loss: 1.0982
Epoch 2/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 12s 101ms/step - categorical_accuracy: 0.3418 - loss: 1.0988 - val_categorical_accuracy: 0.3253 - val_loss: 1.0990
Epoch 3/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 19s 90ms/step - categorical_accuracy: 0.3409 - loss: 1.0989 - val_categorical_accuracy: 0.3253 - val_loss: 1.0987
Epoch 4/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 11s 90ms/step - categorical_accuracy: 0.3453 - loss: 1.0986 - val_categorical_accuracy: 0.3253 - val_loss: 1.0987
Epoch 5/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 11s 94ms/step - categorical_accuracy: 0.3482 - loss: 1.0982 - val_categorical_accuracy: 0.3253 - val_loss: 1.0985
Epoch 6/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 21s 98ms/step - categorical_accuracy: 0.3348 - loss: 1.0983 - val_categorical_accuracy: 0.3253 - val_loss: 1.0989
Epoch 7/50
119/119 ━━━━━━━━━━━━━━━━━━━━ 12s 102ms/step - categorical_accur

In [69]:
pred = model_clf.predict(X_test)
pred

38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 70ms/step


array([[0.32419068, 0.34285554, 0.33295378],
       [0.32419068, 0.34285554, 0.33295378],
       [0.32419068, 0.34285554, 0.33295378],
       ...,
       [0.32419068, 0.34285554, 0.33295378],
       [0.32419068, 0.34285554, 0.33295378],
       [0.32419068, 0.34285554, 0.33295378]], dtype=float32)

In [70]:
pred_ar = [np.argmax(prediction) for prediction in pred]
y_test_ar = [np.argmax(y) for y in y_test]
